*Project Aim*:-

The goal is to:

**Allocate exam seating:**
Assign students to rooms for exams while adhering to constraints like room capacity, seating density (dense or sparse), and buffer seats.


**Generate attendance sheets: **
For each room, create an attendance sheet with the roll numbers and names of the students allocated to that room, with spaces for invigilator and TA signatures.



*Logic Used*
**Data Organization:**

Load data about students, exam schedules, rooms, and roll-to-name mapping from an Excel file.
Organize it into tables (DataFrames) for easy manipulation.


**Seating Allocation:**

Assign students to rooms starting with Block 9, prioritizing lower floors, then allocate to LT halls if there’s an overflow.


**Consider seating modes:**

Dense: Use maximum room capacity, minus the buffer.
Sparse: Use half the room capacity, minus the buffer.
Allocate the largest courses first to minimize room fragmentation.


**Attendance Sheets:**

Create a detailed Excel sheet for each room, showing:
Roll numbers.
Names (mapped from roll numbers).
Blank spaces for signatures.


Step 1: Import Libraries
We need specific Python libraries to handle data, Excel files, and formatting.

**pandas**: Handles tabular data (student data, timetable, etc.).
openpyxl:
Creates and formats Excel files.
Border and Alignment:
Add styles to attendance sheets.

In [1]:
# Install openpyxl for handling Excel files
!pip install openpyxl

# Import required libraries
import pandas as pd
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Alignment, Border, Side


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 4.2 MB/s eta 0:00:00


Step 2: Load Data
We load the data from an Excel file. This file contains:

Student registrations (ip_1): Links roll numbers to courses.
Timetable (ip_2): Lists exams, dates, and shifts.
Room data (ip_3): Specifies room capacities and blocks.
Roll-to-name mapping (ip_4): Maps roll numbers to student names.
python
Copy code


In [2]:
def create_attendance_sheet(attendance_df, exam_date, course_code, room_id, shift, roll_to_name_map):
    """
    Create a detailed attendance sheet for the specified room and shift.
    """
    # Add student names to the attendance data
    attendance_df["Student_Name"] = attendance_df["Roll"].map(roll_to_name_map).fillna("Name Not Found")
    attendance_df["Signature"] = ""

    # Add extra rows for invigilators and TA signatures
    extra_rows = pd.DataFrame({"Roll": [""] * 5, "Student_Name": [""] * 5, "Signature": [""] * 5})
    attendance_df = pd.concat([attendance_df, extra_rows], ignore_index=True)

    # Prepare Excel file
    output_file = f"{exam_date.strftime('%d_%m_%Y')}_{course_code}_{room_id}_{shift.lower()}.xlsx"
    workbook = Workbook()
    sheet = workbook.active
    sheet.title = f"Room {room_id}"

    # Add attendance data to the sheet
    for row in dataframe_to_rows(attendance_df, index=False, header=True):
        sheet.append(row)

    # Adjust column widths
    sheet.column_dimensions["A"].width = 15
    sheet.column_dimensions["B"].width = max(15, attendance_df["Student_Name"].str.len().max() + 2)
    sheet.column_dimensions["C"].width = 20

    # Style the sheet
    border_style = Border(
        left=Side(style="thin"),
        right=Side(style="thin"),
        top=Side(style="thin"),
        bottom=Side(style="thin")
    )
    for row in sheet.iter_rows():
        for cell in row:
            cell.alignment = Alignment(horizontal="center", vertical="center")
            cell.border = border_style

    # Save the Excel file
    workbook.save(output_file)
    print(f"Attendance sheet saved: {output_file}")


Step 3: Allocate Seating
Here, we allocate rooms for each course based on:

Priority:
Start with Block 9, filling lower floors first.
If Block 9 is full, move to LT halls.
Seating Mode:
Adjust maximum capacity based on the buffer and seating mode (dense/sparse).

In [3]:
def load_data(file_path):
    """
    Load data from an Excel file.
    """
    xls = pd.ExcelFile(file_path)
    student_df = pd.read_excel(xls, sheet_name="ip_1", skiprows=1)
    timetable_df = pd.read_excel(xls, sheet_name="ip_2", skiprows=1)
    room_df = pd.read_excel(xls, sheet_name="ip_3")
    roll_to_name_df = pd.read_excel(xls, sheet_name="ip_4")

    return student_df, timetable_df, room_df, roll_to_name_df


Step 4: Create Attendance Sheets
For each room, generate an attendance sheet with:

Student Details:
Roll numbers and names.
Spaces for Signatures:
Blank rows for invigilator and TA signatures.
python
Copy code



In [4]:
def get_max_capacity(room_capacity, seating_mode, buffer_seats):
    """
    Calculates the maximum capacity of a room based on seating mode and buffer seats.
    """
    if seating_mode == "dense":
        return room_capacity - buffer_seats
    elif seating_mode == "sparse":
        return (room_capacity - buffer_seats) // 2  # Assume sparse seating uses half the capacity
    else:
        return room_capacity - buffer_seats  # Default to dense seating if mode is invalid


def allocate_seating(student_df, timetable_df, room_df, buffer_seats, seating_mode):
    """
    Allocate rooms to courses based on student count, block priority, and seating mode.
    """
    timetable_df["Date"] = pd.to_datetime(timetable_df["Date"], dayfirst=True)

    # Separate rooms by block
    block_9 = room_df[room_df["Block"] == 9].sort_values(by=["Room No."])
    lt_rooms = room_df[room_df["Block"] == "LT"].sort_values(by="Exam Capacity", ascending=False)

    course_distribution = student_df.groupby("course_code")["rollno"].count().reset_index()
    course_distribution.columns = ["course_code", "student_count"]
    course_to_students = student_df.groupby("course_code")["rollno"].apply(list).to_dict()

    arrangement = []

    for _, session in timetable_df.iterrows():
        for time in ["Morning", "Evening"]:
            if pd.isna(session[time]):
                continue

            courses = session[time].split("; ")
            course_sizes = {
                course: course_distribution[course_distribution["course_code"] == course]["student_count"].values[0]
                for course in courses
                if course in course_distribution["course_code"].values
            }
            sorted_courses = sorted(course_sizes.items(), key=lambda x: x[1], reverse=True)

            for course, student_count in sorted_courses:
                allocated_students = 0
                assigned_rooms = []

                # Assign rooms in Block 9
                for _, room in block_9.iterrows():
                    if allocated_students >= student_count:
                        break
                    room_capacity = room["Exam Capacity"]
                    max_capacity = get_max_capacity(room_capacity, seating_mode, buffer_seats)
                    allocation = min(max_capacity, student_count - allocated_students)
                    allocated_students += allocation
                    assigned_rooms.append((room["Room No."], allocation))

                # Assign rooms in LT halls if needed
                for _, room in lt_rooms.iterrows():
                    if allocated_students >= student_count:
                        break
                    room_capacity = room["Exam Capacity"]
                    max_capacity = get_max_capacity(room_capacity, seating_mode, buffer_seats)
                    allocation = min(max_capacity, student_count - allocated_students)
                    allocated_students += allocation
                    assigned_rooms.append((room["Room No."], allocation))

                # Record allocation details
                for room_id, allocated in assigned_rooms:
                    students = course_to_students[course][:allocated]
                    course_to_students[course] = course_to_students[course][allocated:]
                    arrangement.append({
                        "Date": session["Date"],
                        "Shift": time,
                        "Course_Code": course,
                        "Room": room_id,
                        "Allocated_Students": allocated,
                        "Students": "; ".join(students)
                    })

    return pd.DataFrame(arrangement)



def create_attendance_sheets(final_arrangement_df, roll_to_name_dict):
    """Generate attendance sheets for each room and shift."""
    # Border styling for Excel output
    cell_border = Border(
        left=Side(style="thin"),
        right=Side(style="thin"),
        top=Side(style="thin"),
        bottom=Side(style="thin")
    )

    # Create attendance sheet per room and shift
    for _, seating_row in final_arrangement_df.iterrows():
        exam_date = seating_row["Date"]
        shift = seating_row["Shift"]
        course_code = seating_row["Course_Code"]
        room_number = seating_row["Room"]
        students = seating_row["Students"].split(";")

        # Create attendance DataFrame
        attendance_data = pd.DataFrame({"Roll": students})
        attendance_data["Roll"] = attendance_data["Roll"].str.strip()
        attendance_data["Name"] = attendance_data["Roll"].map(roll_to_name_dict).fillna("Unknown Name")
        attendance_data["Signature"] = ""

        # Add empty rows for invigilator and TAs' signature at the end
        signature_rows = pd.DataFrame({
            "Roll": ["Invigilator", "TA 1", "TA 2"],
            "Name": ["", "", ""],
            "Signature": ["", "", ""]
        })
        attendance_data = pd.concat([attendance_data, signature_rows], ignore_index=True)

        # Generate Excel file for each room and shift
        file_name = f"{exam_date.strftime('%d_%m_%Y')}_{course_code}_{room_number}_{shift.lower()}.xlsx"

        wb = Workbook()
        ws = wb.active
        ws.title = f"{course_code} Room {room_number}"

        # Write data to the sheet
        for row in dataframe_to_rows(attendance_data, index=False, header=True):
            ws.append(row)

        # Set column widths based on name length
        max_name_len = attendance_data["Name"].str.len().max()
        ws.column_dimensions["B"].width = max(15, max_name_len + 2)

        # Apply formatting
        for row in ws.iter_rows():
            for cell in row:
                cell.alignment = Alignment(horizontal="center", vertical="center")
                cell.border = cell_border

        wb.save(file_name)

    print("Attendance sheets generated successfully.")


Step 5: Combine Everything
Combine the components into a single function for execution.

python
Copy code


In [ ]:
def main():
    # Load the input file
    file_path = "/content/py_project.xlsx"
    student_df, timetable_df, room_df, roll_to_name_df = load_data(file_path)

    # Prompt user for configuration
    buffer_seats = int(input("Enter the number of buffer seats per room: "))
    seating_mode = input("Select seating mode ('dense' or 'sparse'): ").strip().lower()
    if seating_mode not in ["dense", "sparse"]:
        seating_mode = "dense"
        print("Invalid mode. Defaulting to 'dense'.")

    # Roll number to name mapping
    roll_to_name_map = dict(zip(roll_to_name_df["Roll"].astype(str), roll_to_name_df["Name"]))

    # Allocate seating
    arrangement_df = allocate_seating(student_df, timetable_df, room_df, buffer_seats, seating_mode)
    arrangement_df.to_excel("seating_arrangement.xlsx", index=False)
    print("Seating arrangement saved to 'seating_arrangement.xlsx'.")

    # Generate attendance sheets
    for _, row in arrangement_df.iterrows():
        students = row["Students"].split("; ")
        attendance_df = pd.DataFrame({"Roll": students})
        create_attendance_sheet(
            attendance_df,
            row["Date"],
            row["Course_Code"],
            row["Room"],
            row["Shift"],
            roll_to_name_map
        )
    print("Attendance sheets generated successfully.")

if __name__ == "__main__":
    main()
